![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [28]:
# Import necessary libraries
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


# Preprocess the data

## Preprocess "object" type columns

In [29]:
# check columns data type
cc_apps.dtypes

0      object
1      object
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12      int64
13     object
dtype: object

In [30]:
# copy dataframe to work with the new one
cc_apps_prep = cc_apps.copy()

In [31]:
# Inspect "?" value in the third column as a substitute for the other 'object' columns
cc_apps_prep[3].value_counts()

u    519
y    163
?      6
l      2
Name: 3, dtype: int64

In [32]:
# replace "?" with null for object type columns
for column in cc_apps_prep.columns:
    if cc_apps_prep[column].dtype == 'object':
        cc_apps_prep[column] = cc_apps_prep[column].replace("?", np.nan)

In [33]:
# Check values after replace "?" with null
cc_apps_prep[3].value_counts()

u    519
y    163
l      2
Name: 3, dtype: int64

In [34]:
# The thrid column has 6 null values which is the same number of "?" before cleaning
cc_apps_prep[3].isna().sum()

6

In [35]:
# replace null with the most frequent value for each 'object' type column
for column in cc_apps_prep.columns:
    if cc_apps_prep[column].dtype == 'object':
        # get the most frequent value in the column
        most_frequent_value = cc_apps_prep[column].value_counts().index[0]
        cc_apps_prep[column].fillna(most_frequent_value, inplace=True)

In [36]:
# Check for null values
cc_apps_prep[3].isna().sum()

0

In [37]:
# Check if null values has been changed into most frequent value
cc_apps_prep[3].value_counts()

u    525
y    163
l      2
Name: 3, dtype: int64

## Preprocess numerical columns

In [38]:
# replace missing values with the mean
for column in cc_apps_prep.columns:
    if cc_apps_prep[column].dtype == 'float64' or cc_apps_prep[column].dtype == 'int64':
        # get the mean value in the column
        mean = cc_apps_prep[column].mean()
        cc_apps_prep[column].fillna(mean, inplace=True)

In [39]:
# One-hot encoding
cc_apps_dummies = pd.get_dummies(cc_apps_prep, drop_first=True)
cc_apps_dummies.columns

Index([        2,         7,        10,        12,     '0_b', '1_15.17',
       '1_15.75', '1_15.83', '1_15.92', '1_16.00',
       ...
           '6_j',     '6_n',     '6_o',     '6_v',     '6_z',     '8_t',
           '9_t',    '11_p',    '11_s',    '13_-'],
      dtype='object', length=383)

In [40]:
cc_apps_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Columns: 383 entries, 2 to 13_-
dtypes: float64(2), int64(2), uint8(379)
memory usage: 277.1 KB


# Prepare the data for modeling

In [41]:
# Create explanatory and response variables
X = cc_apps_dummies.iloc[:, :-1].values
y = cc_apps_dummies.iloc[:, -1].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Apply standard scaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a model

In [42]:
# Create Logistic Regression object, fit and predict the data
reg = LogisticRegression()
reg_scaled = reg.fit(X_train_scaled, y_train)
y_pred = reg_scaled.predict(X_test_scaled)
print(reg_scaled.score(X_test_scaled, y_test))

0.8188405797101449


In [43]:
# Add accuracy score to 'best_score' variable to summit the project
best_score = reg_scaled.score(X_test_scaled, y_test)